<a href="https://colab.research.google.com/github/Sabina-0219/Sentiment-Analysis-System/blob/main/Sentiment_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import urllib.request
import os
import tarfile
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import re
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Flatten

Using TensorFlow backend.


In [3]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar zxf aclImdb_v1.tar.gz

--2022-07-02 08:45:33--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  60.2MB/s    in 1.3s    

2022-07-02 08:45:34 (60.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [4]:
def rm_tags(text):
  re_tag = re.compile(r'<[^>]+>')
  return re_tag.sub('', text)

In [5]:
def read_files(filetype):
  path = "/content/aclImdb/"
  file_list = []

  positive_path = path + filetype + "/pos/"
  for f in os.listdir(positive_path):
    file_list += [positive_path+f]
  
  negative_path = path + filetype + "/neg/"
  for f in os.listdir(negative_path):
    file_list += [negative_path+f]
  
  print("read", filetype, "files:", len(file_list))

  all_labels = ([1] * 12500 + [0] * 12500)

  all_texts = []
  for fi in file_list:
    with open(fi, encoding = 'utf8') as file_input:
      all_texts += [rm_tags(" ".join(file_input.readlines()))]
  return all_labels, all_texts

In [6]:
y_train, train_text = read_files("train")

read train files: 25000


In [7]:
y_test, test_text = read_files("test")

read test files: 25000


In [8]:
train_text[0]

"I saw this series when I was a kid and loved the detail it went into and never forgot it. I finally purchased the DVD collection and its just how I remembered. This is just how a doco should be, unbiased and factual. The film footage is unbelievable and the interviews are fantastic. The only other series that I have found equal to this is 'Die Deutschen Panzer'.I only wish Hollywood would sit down and watch this series, then they might make some great war movies.Note. Band of Brothers, Saving Private Ryan, Letters from Iwo Jima, Flags of Our Fathers and When Trumpets Fade are some I'd recommend"

In [9]:
y_train[0]

1

In [10]:
train_text[20000]

'This is one of the most irritating, nonsensical movies I\'ve ever had the misfortune to sit through. Every time it started to look like it might be getting good, out come more sepia tone flashbacks, followed by paranoid idiocy masquerading as social commentary. The main character, Maddox, is a manipulative, would-be rebel who lives in a mansion seemingly without any parents or responsibility. The supporting cast are all far more likeable and interesting, but are unfortunately never developed. Nor do we ever really understand the John Stanton character supposedly influencing Maddox to commit the acts of rebellion. At one point, I thought "Aha! Maddox is just nuts and is secretly making up all those communications from escaped mental patient Stanton! Now we\'re getting somewhere!" but of course, that ends up to not be the case and the whole movie turns out to be pointless, both from Maddox\'s perspective and the viewer\'s. Where\'s Ferris Bueller when we need him?'

In [11]:
y_train[20000]

0